# Assignment 1: K-Nearest Neighbors Classification (15 marks)

Student Name: Jiayi Li

Student ID: 1097419

## General info

<b>Due date</b>: Friday, 19 March 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 15% of mark for class. 

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/105477/pages/python-and-jupyter-notebooks?module_item_id=2613813) on Canvas (under Modules> Coding Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  


<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it. We reserve the right to deduct up to 2 marks for unreadable or exessively inefficient code.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board (Piazza -> Assignment_1); we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. Please check the <a href="https://canvas.lms.unimelb.edu.au/courses/105477/modules">CIS Academic Honesty training</a> for more information. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this assignment, you will implement the K-nearest neighbor (KNN) classification algorithm and apply it to a real-world machine learning data set. In particular we will classify zoo animals into seven animal categories. 

Firstly, you will read in the dataset into a train and a test set (Q1). Secondly, you will implement different distance functions (Q2). Thirdly, you will implement a KNN classifier (Q3). You will apply and evaluate your classifier on the data set exploring different parameters (Q4, Q5). Finally, you will critically discuss your results (Q6).

### Question 1: Loading the data (2.0 marks)

**Instructions:** For this assignment we will develop a K-Nearest Neighbors (KNN) classifier to classify animals in the zoo into pre-defined categories of animals, namely
```
1: mammal
2: bird
3: reptile
4: fish
5: amphibian
6: insect
7: invertebrate
```

We use a data set of zoo animals from the UCI Machine learning repository.

The original data can be found here: https://archive.ics.uci.edu/ml/datasets/Zoo. 

The dataset consists of 101 instances. Each instance corresponds to an animal which has a unique identifier (the name of the animal; first field) and is characterized with 16 features:

```
   1. hair		    Boolean
   2. feathers		Boolean
   3. eggs		    Boolean
   4. milk		    Boolean
   5. airborne		Boolean
   6. aquatic		Boolean
   7. predator		Boolean
   8. toothed		Boolean
   9. backbone		Boolean
  10. breathes		Boolean
  11. venomous		Boolean
  12. fins		    Boolean
  13. legs		    Numeric (set of values: {0,2,4,5,6,8})
  14. tail		    Boolean
  15. domestic		Boolean
  16. catsize		Boolean
```

You need to first obtain this dataset, which is on Canvas (assignment 1). The files we will be using are called *zoo.data* and *zoo.labels*. Make sure the files are saved in the same folder as this notebook.

Both files are in comma-separated value format.

*zoo.features* contains 101 instances, one line per instance. The first field is the unique instance identifier (name of animals). The following fields contain the 16 features, as described above.

*zoo.labels* contains the gold labels (i.e., one of the seven classes above), for one instance per line. Again, the first field is the instance identifier, and the second field the instance label.

**Task**: Read the two files  
1. create a **training_feature** set (list of features for the first 90 instances in the zoo.* files) and a **training_label** set (list of labels for the corresponding). 
2. create a **test_feature** set (list of features of the remaining instances in the zoo.* files) and a **test_label** set (list of labels for the corresponding). 

**Check**: Use the assertion statements in *"For your testing"* to validate your code.

In [283]:
data = open("zoo.features", 'r').readlines()
gold = open("zoo.labels", 'r').readlines()

train_features = []
train_labels   = []
test_features = []
test_labels   = []

# iterate over each instance, turn string features into number features, store them as train/test_features
for instance in data:
    inst_format = instance.strip().split(',') 
    num_features = [] 
    
    # refered from: https://www.kite.com/python/answers/how-to-extract-integers-from-a-string-in-python
    for word in inst_format:
        if word.isdigit():
            num_features.append(int(word))
            
    if data.index(instance) < 90:
        train_features.append(num_features)
    else:
        test_features.append(num_features)

# interate over each label, format them, put them into train/test_labels
for label in gold:
    label_format = label.strip().split(',')
    inst_label = label_format[1]
    
    if gold.index(label) < 90:
        train_labels.append(int(inst_label))
    else:
        test_labels.append(int(inst_label))
        

# Reference for this question: The University of Melbourne,(2021) Week1_demo_1.ipynb (Version 1.0) [Source code]. https://canvas.lms.unimelb.edu.au/courses/105477/files/6873411?wrap=1

In [284]:
print(train_features)      
print(test_features)

[[1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 4, 0, 0, 1], [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 4, 1, 0, 1], [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 4, 0, 0, 1], [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 4, 1, 0, 1], [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 4, 1, 0, 1], [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 4, 1, 1, 1], [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0], [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 4, 0, 1, 0], [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 4, 1, 0, 1], [0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 2, 1, 1, 0], [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0], [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 6, 0, 0, 0], [0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 4, 1, 0, 1], [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1], [0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1],

In [285]:
print(train_labels)
print(test_labels)

[1, 1, 4, 1, 1, 1, 1, 4, 4, 1, 1, 2, 4, 7, 7, 7, 2, 1, 4, 1, 2, 2, 1, 2, 6, 5, 5, 1, 1, 1, 6, 1, 1, 2, 4, 1, 1, 2, 4, 6, 6, 2, 6, 2, 1, 1, 7, 1, 1, 1, 1, 6, 5, 7, 1, 1, 2, 2, 2, 2, 4, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 7, 4, 1, 1, 3, 7, 2, 2, 3, 7, 4, 2, 1, 7, 4, 2, 6, 5]
[3, 3, 4, 1, 1, 2, 1, 6, 1, 7, 2]


<b>For your testing</b>

In [286]:
assert len(train_features) == len(train_labels)
assert len(train_features[0])==len(train_features[-1])
assert train_features[2][12]==0 and train_labels[2]==4

### Question 2: Distance Functions (2.0 marks)

<b>Instructions</b>: Implement the four distance functions specified below. Use <b>only</b> the library imported below, i.e., <b>do not</b> use implementations from any other Python library. 

1. Eucledian distance
2. Cosine distance
3. Hamming distance 
4. Jaccard distance 

Each distance function takes as input
- Two feature vectors

and returns as output
- The distance between the two feature vectors (float)

**Note** for the purpose of this assignment we consider the numeric feature (legs) to be discretized already with each individual value belonging constituting a separate class.

In [287]:
import math

# assume fv1 is features of each instance from test_features, while fv2 is those from train_features
   
# refered by: The University of Melbourne,(2021) w2-solution-21s1.ipynb (Version 1.0) [Source code]. https://canvas.lms.unimelb.edu.au/courses/105477/files/6869589?wrap=1
def eucledian_distance(fv1, fv2): 
    return math.sqrt(sum([(fv1[i]-fv2[i])*(fv1[i]-fv2[i]) for i in range(len(fv1))]))
    
# refered by: The University of Melbourne,(2021) w2-solution-21s1.ipynb (Version 1.0) [Source code]. https://canvas.lms.unimelb.edu.au/courses/105477/files/6869589?wrap=1
def cosine_distance(fv1, fv2):
    return sum([fv1[i]*fv2[i] for i in range(len(fv1))]) / math.sqrt(sum([pow(value,2) for value in fv1])) * math.sqrt(sum([pow(value,2) for value in fv2]))
        

def hamming_distance(fv1, fv2):
    ham_dist = 0
    for i in range(len(fv1)):
        if fv1[i] != fv2[i]:
            ham_dist += 1        
    return ham_dist
    

def jaccard_distance(fv1, fv2):
    fv1_feature_num = 0
    fv2_feature_num = 0
    intersection = 0
    
    for value in fv1:
        if value == 1:
            fv1_feature_num += 1
            
    for value in fv2:
        if value == 1:
            fv2_feature_num += 1

    for i in range(len(fv1)):
        if fv1[i] == fv2[i] == 1:
            intersection += 1
            
    return 1 - (intersection / (fv1_feature_num + fv2_feature_num - intersection))

<b>For your testing: </b>

In [288]:
assert round(eucledian_distance([1,0],[0.5,1]),2)==1.12 
assert jaccard_distance([1,1,1,1], [0,1,0,0])==0.75

### Question 3: KNN Classifier (4.0 marks)

<b>Instructions</b>: Here, you implement your KNN classifier. It takes as input 
- training data features
- training data labels
- test data features
- parameter K
- distance function(s) based on which nearest neighbors will be identified

It returns as output 
- the predicted labels for the test data

**Voting stragegy** Your classifier will use majority voting (i.e., no distance weighting)

**You should implement the classifier from scratch yourself**, i.e., <b> you must not</b> use an existing implementation in any Python library.

**Ties**. Ties may occur when computing the K nearest neighbors, or when predicting the class based on the neighborhood. You may deal with ties whichever way you want (as long as you still use the requested distance functions).

In [289]:
import numpy as np
from scipy import stats


def KNN(train_features, train_labels, test_features, k, dist_fun):
    
    predictions = np.array([], dtype = int)
    
    # calculate distance between each test features and training features
    for test_feat in test_features:
        distances = [] 
        for train_feat in train_features:
            if id(dist_fun) == id(eucledian_distance):
                distances.append(eucledian_distance(test_feat, train_feat))
            if id(dist_fun) == id(cosine_distance):
                distances.append(cosine_distance(test_feat, train_feat))
            if id(dist_fun) == id(jaccard_distance):
                rules_to_convert_legs(test_feat, train_feat)
                distances.append(jaccard_distance(test_feat, train_feat))
            if id(dist_fun) == id(hamming_distance):
                rules_to_convert_legs(test_feat, train_feat)
                distances.append(hamming_distance(test_feat, train_feat))
          
        # combines corresponding train labels and distance between each test feature and training features
        match_labels = np.array(train_labels)
        train_distances = np.array(distances)
            
        labels_dist = np.stack((match_labels, train_distances),axis = -1)
            
        # sort the array to find the first k labels and distance
        # refered from: https://www.kite.com/python/answers/how-to-sort-the-rows-of-a-numpy-array-by-a-column-in-python
        sorted_labels_dist = labels_dist[np.argsort(labels_dist[:,1])]
        KNN = sorted_labels_dist[:k]
        
        KNN_labels = []  
        for match in KNN:
            KNN_labels.append(match[0])
            
        # find the mode in KNN labels, which is the prediction for the test feature
        mode = stats.mode(KNN_labels)[0]
            
        predictions = np.append(predictions, mode)
            
    return predictions  

    
# rules to deal with legs 
# if the number of legs are the same, turn both fv1[12] and fv2[12], which are the position of legs feature, into 1
# otherwise, turn fv1[12] to 0, fv2[12] to 1
def rules_to_convert_legs(fv1, fv2):
    if fv1[12] == fv2[12]:
        fv1[12] == 1
        fv2[12] == 1
    else:
        fv1[12] == 0
        fv2[12] == 1

<b>For your testing:</b>

In [290]:
assert KNN([[1,1],[5,5],[1,2]], [1,0,1], [[1,1]], 1, eucledian_distance) == [1]

### Question 4: Evaluation (1.0 marks)
**Instructions:** Write a function that computes the "accuracy" of your classifier. Given a set of predicted lables and a set of gold labels, it returns the fraction of correct labels over all predicted labels. 

**Example**: `The gold truth labels for four test instances are [1, 1, 1, 1]. A system predicted the labels [0, 1, 0, 0] for the same 4 instances. The accuracy of the system is 1/4 = 0.25
`

Your function will take as input 
- gold labels
- predicted labels

It returns as output
- the accuracy value (float).


In [291]:
def accuracy(predict, gold):
    predict = np.array(predict)
    gold = np.array(gold)
    num_matched_label = 0
 
    for i in range(len(predict)):
        if predict[i] == gold[i]:
            num_matched_label += 1
    
    accuracy = round((num_matched_label / len(predict)),2)

    return accuracy

<b>For your testing:</b>

In [292]:
assert accuracy([1, 1, 1, 1], [0, 1, 0, 1])==0.5

### Question 5: Applying your KNN classifiers to the Zoo Dataset (3.0 marks)

**Using the functions you have implemented above, please**

<b> 1. </b>
For each of the distance functions you implemented in Question 2, construct four KNN classifiers with K=1, K=5, K=25, K=55. You will create a total of 16 (4 distance functions x 4 K values) classifiers.

<b> 2. </b>
Compute the test accuracy for each model

In [293]:
# 1. Predict test labels with each KNN classifier
classifier1 = KNN(train_features, train_labels, test_features, 1, eucledian_distance)
classifier2 = KNN(train_features, train_labels, test_features, 5, eucledian_distance)
classifier3 = KNN(train_features, train_labels, test_features, 25, eucledian_distance)
classifier4 = KNN(train_features, train_labels, test_features, 55, eucledian_distance)

classifier5 = KNN(train_features, train_labels, test_features, 1, cosine_distance)
classifier6 = KNN(train_features, train_labels, test_features, 5, cosine_distance)
classifier7 = KNN(train_features, train_labels, test_features, 25, cosine_distance)
classifier8 = KNN(train_features, train_labels, test_features, 55, cosine_distance)

classifier9 = KNN(train_features, train_labels, test_features, 1, jaccard_distance)
classifier10 = KNN(train_features, train_labels, test_features, 5, jaccard_distance)
classifier11 = KNN(train_features, train_labels, test_features, 25, jaccard_distance)
classifier12 = KNN(train_features, train_labels, test_features, 55, jaccard_distance)

classifier13 = KNN(train_features, train_labels, test_features, 1, hamming_distance)
classifier14 = KNN(train_features, train_labels, test_features, 5, hamming_distance)
classifier15 = KNN(train_features, train_labels, test_features, 25, hamming_distance)
classifier16 = KNN(train_features, train_labels, test_features, 55, hamming_distance)

# 2. Compute the accuracy scores 
accuracy_knn_euc_1 = accuracy(classifier1, test_labels)
accuracy_knn_euc_5 = accuracy(classifier2, test_labels)
accuracy_knn_euc_25 = accuracy(classifier3, test_labels)
accuracy_knn_euc_55 = accuracy(classifier4, test_labels)

accuracy_knn_cos_1 = accuracy(classifier5, test_labels)
accuracy_knn_cos_5 = accuracy(classifier6, test_labels)
accuracy_knn_cos_25 = accuracy(classifier7, test_labels)
accuracy_knn_cos_55 = accuracy(classifier8, test_labels)

accuracy_knn_jac_1 = accuracy(classifier9, test_labels)
accuracy_knn_jac_5 = accuracy(classifier10, test_labels)
accuracy_knn_jac_25 = accuracy(classifier11, test_labels)
accuracy_knn_jac_55 = accuracy(classifier12, test_labels)

accuracy_knn_ham_1 = accuracy(classifier13, test_labels)
accuracy_knn_ham_5 = accuracy(classifier14, test_labels)
accuracy_knn_ham_25 = accuracy(classifier15, test_labels)
accuracy_knn_ham_55 = accuracy(classifier16, test_labels)


print("Euclidean")
print(accuracy_knn_euc_1)
print(accuracy_knn_euc_5)
print(accuracy_knn_euc_25)
print(accuracy_knn_euc_55)
print("Cosine")
print(accuracy_knn_cos_1)
print(accuracy_knn_cos_5)
print(accuracy_knn_cos_25)
print(accuracy_knn_cos_55)
print("Jaccard")
print(accuracy_knn_jac_1)
print(accuracy_knn_jac_5)
print(accuracy_knn_jac_25)
print(accuracy_knn_jac_55)
print("Hamming")
print(accuracy_knn_ham_1)
print(accuracy_knn_ham_5)
print(accuracy_knn_ham_25)
print(accuracy_knn_ham_55)

Euclidean
0.82
0.82
0.64
0.36
Cosine
0.0
0.0
0.0
0.18
Jaccard
0.82
0.82
0.64
0.45
Hamming
0.82
0.82
0.73
0.45


### Question 6: Discussion (3.0 marks)
1. (a) Which parameter K resulted in the best performance? (b) Why? (c) What could be done to improve those classifiers that are currently performing poorly?

2. The results of KNN with Euclidean distance and KNN with Cosine distance are remarkably similar. Why is that so? Referring to the definitions of the distance functions.

<b>Each question should be answered in no more than 2-3 sentences.</b>

1(a) K = 1 or K = 5 resulted in tbe best performance

1(b) The smaller K means the narrower scope of neighbours to the test instance, which circles the K most similar/least different train instances around irrespective of which approach to calculate distances. So the result with smaller K would be better.

1(c) For the Cosines Distance, maybe we could replace features by various numerical values so the results would be better. Boolean values would cause dramatical bias since the multiplication of Cosine Distance would be significantly affected by zeros, causing each training instance is super close to the test instance. So the program doesn't know which training instance is exactly similar to the test instance. Numerical data would better help Cosine Distance to normalize the magnitude, then find the reasonable KNN to decide the classifier.

2 My results of this two approaches are quite different, since I keep the numbers in legs to calculate. But Euclidean Distance mainly uses abstraction and addition so the distance was not affected a lot, while Cosines Distance uses only multiplication so the effect of magnitude normalization would be offset.
